In [1]:
import requests
import json

# Define the URL and the data payload
url = "http://localhost:11434/api/generate"
payload = {
    "model": "llama3.1",
    "prompt": "who is the president of USA",
    "stream": False
}

# Set the headers
headers = {
    'Content-Type': 'application/json'
}

# Make the POST request
response = requests.post(url, headers=headers, data=json.dumps(payload))

# Print the response
print(response.json())


{'model': 'llama3.1', 'created_at': '2024-07-31T13:24:40.1172474Z', 'response': 'As of my last update in April 2023, the President of the United States is Joe Biden. He took office on January 20, 2021, succeeding Donald Trump. Prior to becoming President, Joe Biden served as Vice President under Barack Obama from 2009 to 2017.', 'done': True, 'done_reason': 'stop', 'context': [128006, 882, 128007, 271, 14965, 374, 279, 4872, 315, 7427, 128009, 128006, 78191, 128007, 271, 2170, 315, 856, 1566, 2713, 304, 5936, 220, 2366, 18, 11, 279, 4900, 315, 279, 3723, 4273, 374, 13142, 38180, 13, 1283, 3952, 5274, 389, 6186, 220, 508, 11, 220, 2366, 16, 11, 73820, 9641, 3420, 13, 32499, 311, 10671, 4900, 11, 13142, 38180, 10434, 439, 23270, 4900, 1234, 24448, 7250, 505, 220, 1049, 24, 311, 220, 679, 22, 13], 'total_duration': 51249649700, 'load_duration': 21295352300, 'prompt_eval_count': 16, 'prompt_eval_duration': 3972117000, 'eval_count': 61, 'eval_duration': 25964875000}


In [1]:
import requests
import json

# Class to interact with the Llama model server
class LlamaLLM:
    def __init__(self, url):
        self.url = url

    def generate(self, prompt):
        payload = {
            "model": "llama3.1",
            "prompt": prompt,
            "stream": False
        }
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.post(self.url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        return response.json()['response']

# Environment class remains the same
class Environment:
    def __init__(self):
        self.state = 'initial_state'  # Initial state
        self.goal_achieved = False  # Goal not achieved
        self.result = None  # Store result

    def get_state(self):
        return self.state  # Get current state

    def change_state(self, result):
        self.state = 'goal_state'  # Change to goal state
        self.result = result  # Store result

# ReActAgent class modified to use LlamaLLM for reasoning
class ReActAgent:
    def __init__(self, environment, llm, max_attempts=5):
        self.environment = environment  # Initialize environment
        self.llm = llm  # Llama 3.1 model
        self.max_attempts = max_attempts  # Max attempts

    def perceive(self):
        return self.environment.get_state()  # Perceive environment state

    def reason(self, state):
        # Causal reasoning prompt based on paper example
        prompt = """
        Given the following causal relationships:
        - A causes B
        - B causes C
        - A is independent of C given B
        
        Question: Does A directly affect C?
        """
        response = self.llm.generate(prompt).strip()
        print(f"Raw Llama Response: {response}")  # Debugging: print raw response
        if 'No' in response or 'independent' in response:
            return 'achieve_goal', "A does not directly affect C"
        return 'take_action', None

    def act(self, action, result):
        if action == 'achieve_goal':
            self.environment.goal_achieved = True  # Achieve goal
            self.environment.change_state(result)  # Change state and store result

# Initialize the Llama model with the server URL
llm = LlamaLLM(url="http://localhost:11434/api/generate")

# Create environment and agent
env = Environment()
agent = ReActAgent(env, llm)

# Agent perception, reasoning, and action loop
attempts = 0
while not env.goal_achieved and attempts < agent.max_attempts:
    state = agent.perceive()
    action, result = agent.reason(state)
    agent.act(action, result)
    attempts += 1
    print(f"State: {state}, Action: {action}, Result: {result}, Attempts: {attempts}")

if env.goal_achieved:
    print(f"Goal achieved with result: {env.result}")
else:
    print("Failed to achieve goal within the maximum number of attempts.")


Raw Llama Response: Based on the information provided:

1. A causes B (A → B)
2. B causes C (B → C)
3. A is independent of C given B (A ⊥ C | B)

The question asks whether A directly affects C.

Since A is independent of C given B, this implies that any effect of A on C must be mediated through B. In other words, the relationship between A and C is indirect, not direct.

Therefore, the answer is: **No**, A does not directly affect C.
State: initial_state, Action: achieve_goal, Result: A does not directly affect C, Attempts: 1
Goal achieved with result: A does not directly affect C


In [2]:
import requests
import json

# Class to interact with the Llama model server
class LlamaLLM:
    def __init__(self, url):
        self.url = url

    def generate(self, prompt):
        payload = {
            "model": "llama3.1",
            "prompt": prompt,
            "stream": False
        }
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.post(self.url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        return response.json()['response']

# Environment class remains the same
class Environment:
    def __init__(self):
        self.state = 'initial_state'  # Initial state
        self.goal_achieved = False  # Goal not achieved
        self.result = None  # Store result

    def get_state(self):
        return self.state  # Get current state

    def change_state(self, result):
        self.state = 'goal_state'  # Change to goal state
        self.result = result  # Store result

# ReActAgent class modified to use LlamaLLM for reasoning
class ReActAgent:
    def __init__(self, environment, llm, max_attempts=5):
        self.environment = environment  # Initialize environment
        self.llm = llm  # Llama 3.1 model
        self.max_attempts = max_attempts  # Max attempts

    def perceive(self):
        return self.environment.get_state()  # Perceive environment state

    def reason(self, state):
        # Causal reasoning prompt based on paper example
        prompt = """
        Given the following causal relationships in a healthcare context:
        - Smoking causes Lung Cancer
        - Lung Cancer causes Mortality
        - Smoking is independent of Mortality given Lung Cancer
        
        Question: Does Smoking directly affect Mortality?
        """
        response = self.llm.generate(prompt).strip()
        print(f"Raw Llama Response: {response}")  # Debugging: print raw response
        if 'No' in response or 'independent' in response:
            return 'achieve_goal', "Smoking does not directly affect Mortality"
        return 'take_action', None

    def act(self, action, result):
        if action == 'achieve_goal':
            self.environment.goal_achieved = True  # Achieve goal
            self.environment.change_state(result)  # Change state and store result

# Initialize the Llama model with the server URL
llm = LlamaLLM(url="http://localhost:11434/api/generate")

# Create environment and agent
env = Environment()
agent = ReActAgent(env, llm)

# Agent perception, reasoning, and action loop
attempts = 0
while not env.goal_achieved and attempts < agent.max_attempts:
    state = agent.perceive()
    action, result = agent.reason(state)
    agent.act(action, result)
    attempts += 1
    print(f"State: {state}, Action: {action}, Result: {result}, Attempts: {attempts}")

if env.goal_achieved:
    print(f"Goal achieved with result: {env.result}")
else:
    print("Failed to achieve goal within the maximum number of attempts.")


Raw Llama Response: A nice example of a directed acyclic graph (DAG)!

Let's analyze the relationships:

1. Smoking → Lung Cancer (causal relationship)
2. Lung Cancer → Mortality (causal relationship)
3. Smoking ⊥ Mortality | Lung Cancer (independence statement, meaning Smoking is independent of Mortality given Lung Cancer)

The question asks: Does Smoking directly affect Mortality?

To answer this, we need to consider the following:

* If there were a direct effect from Smoking to Mortality, it would be represented by an edge from Smoking to Mortality.
* However, since Lung Cancer lies on the path between Smoking and Mortality (Smoking → Lung Cancer → Mortality), any effect of Smoking on Mortality must go through Lung Cancer.

In other words, given that we know there is a causal relationship between Lung Cancer and Mortality, if Smoking influences Mortality, it can only do so through its effect on Lung Cancer. 

Now, the independence statement tells us that Smoking is independent of M

In [3]:
import requests
import json

# Class to interact with the Llama model server
class LlamaLLM:
    def __init__(self, url):
        self.url = url

    def generate(self, prompt):
        payload = {
            "model": "llama3.1",
            "prompt": prompt,
            "stream": False
        }
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.post(self.url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        return response.json()['response']

# Environment class remains the same
class Environment:
    def __init__(self):
        self.state = 'initial_state'  # Initial state
        self.goal_achieved = False  # Goal not achieved
        self.result = None  # Store result

    def get_state(self):
        return self.state  # Get current state

    def change_state(self, result):
        self.state = 'goal_state'  # Change to goal state
        self.result = result  # Store result

# ReActAgent class modified to use LlamaLLM for reasoning
class ReActAgent:
    def __init__(self, environment, llm, max_attempts=5):
        self.environment = environment  # Initialize environment
        self.llm = llm  # Llama 3.1 model
        self.max_attempts = max_attempts  # Max attempts

    def perceive(self):
        return self.environment.get_state()  # Perceive environment state

    def reason(self, state):
        # Causal reasoning prompt based on paper example
        prompt = """
        Given the following causal relationships in a healthcare context:
        - Smoking causes Lung Cancer
        - Lung Cancer causes Mortality
        - Smoking is independent of Mortality given Lung Cancer
        
        Question: Does Smoking directly affect Mortality?
        """
        response = self.llm.generate(prompt).strip()
        print(f"Raw Llama Response: {response}")  # Debugging: print raw response
        if 'No' in response or 'independent' in response:
            return 'achieve_goal', "Smoking does not directly affect Mortality"
        return 'take_action', None

    def act(self, action, result):
        if action == 'achieve_goal':
            self.environment.goal_achieved = True  # Achieve goal
            self.environment.change_state(result)  # Change state and store result

# Initialize the Llama model with the server URL
llm = LlamaLLM(url="http://localhost:11434/api/generate")

# Create environment and agent
env = Environment()
agent = ReActAgent(env, llm)

# Agent perception, reasoning, and action loop
attempts = 0
while not env.goal_achieved and attempts < agent.max_attempts:
    state = agent.perceive()
    action, result = agent.reason(state)
    agent.act(action, result)
    attempts += 1
    print(f"State: {state}, Action: {action}, Result: {result}, Attempts: {attempts}")

if env.goal_achieved:
    print(f"Goal achieved with result: {env.result}")
else:
    print("Failed to achieve goal within the maximum number of attempts.")


Raw Llama Response: Based on the given causal relationships, let's analyze the effect of smoking on mortality:

1. **Smoking causes Lung Cancer**: This establishes a direct relationship between smoking and lung cancer.
2. **Lung Cancer causes Mortality**: This shows that lung cancer has a direct impact on mortality.

However, there is also a statement that **Smoking is independent of Mortality given Lung Cancer**. This means that the effect of smoking on mortality cannot be explained by its association with lung cancer; in other words, once we control for (or "given") lung cancer, smoking does not have any additional effect on mortality.

Now, let's ask: Does Smoking directly affect Mortality?

Since there is a direct relationship between Lung Cancer and Mortality, but no independent effect of Smoking on Mortality given Lung Cancer, the correct answer to this question would be **No**. The direct effect of smoking on mortality is fully mediated by its impact on lung cancer.

In simpler 

In [ ]:
import requests
import json

# Class to interact with the Llama model server
class LlamaLLM:
    def __init__(self, url):
        self.url = url

    def generate(self, prompt):
        payload = {
            "model": "llama3.1",
            "prompt": prompt,
            "stream": False
        }
        headers = {
            'Content-Type': 'application/json'
        }
        response = requests.post(self.url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()
        return response.json()['response']

# Environment class represents the state and goal of the environment
class Environment:
    def __init__(self):
        self.state = 'initial_state'  # Initial state
        self.goal_achieved = False  # Goal not achieved
        self.result = None  # Store result

    def get_state(self):
        return self.state  # Get current state

    def change_state(self, result):
        self.state = 'goal_state'  # Change to goal state
        self.result = result  # Store result

# ReActAgent class uses LlamaLLM for reasoning
class ReActAgent:
    def __init__(self, environment, llm, max_attempts=5):
        self.environment = environment  # Initialize environment
        self.llm = llm  # Llama 3.1 model
        self.max_attempts = max_attempts  # Max attempts

    def perceive(self):
        return self.environment.get_state()  # Perceive environment state

    def reason(self, state):
        # Multi-step causal reasoning prompt based on paper example
        if state == 'initial_state':
            prompt = """
            Given the following causal relationships in a healthcare context:
            - Smoking causes Lung Cancer
            - Lung Cancer causes Mortality
            - Treatment affects Lung Cancer
            - Treatment affects Mortality directly
            
            Question: Does Treatment affect Mortality, considering its effect on Lung Cancer?
            """
        elif state == 'step_1':
            prompt = """
            Based on the causal relationships provided:
            - Smoking causes Lung Cancer
            - Lung Cancer causes Mortality
            - Treatment affects Lung Cancer
            - Treatment affects Mortality directly
            
            Question: What is the direct effect of Treatment on Mortality?
            """
        else:
            prompt = "Invalid state for reasoning."

        response = self.llm.generate(prompt).strip()
        print(f"Raw Llama Response: {response}")  # Debugging: print raw response
        if state == 'initial_state' and ('Yes' in response or 'directly' in response):
            return 'step_1', response
        elif state == 'step_1' and 'direct effect' in response:
            return 'achieve_goal', response
        return 'take_action', None

    def act(self, action, result):
        if action == 'step_1':
            self.environment.state = 'step_1'  # Move to next step
        elif action == 'achieve_goal':
            self.environment.goal_achieved = True  # Achieve goal
            self.environment.change_state(result)  # Change state and store result

# Initialize the Llama model with the server URL
llm = LlamaLLM(url="http://localhost:11434/api/generate")

# Create environment and agent
env = Environment()
agent = ReActAgent(env, llm)

# Agent perception, reasoning, and action loop
attempts = 0
while not env.goal_achieved and attempts < agent.max_attempts:
    state = agent.perceive()
    action, result = agent.reason(state)
    agent.act(action, result)
    attempts += 1
    print(f"State: {state}, Action: {action}, Result: {result}, Attempts: {attempts}")

if env.goal_achieved:
    print(f"Goal achieved with result: {env.result}")
else:
    print("Failed to achieve goal within the maximum number of attempts.")


Raw Llama Response: A nice causal diagram!

Let's break down the relationships:

1. **Smoking → Lung Cancer**: Smoking is a direct cause of Lung Cancer.
2. **Lung Cancer → Mortality**: Having Lung Cancer directly increases the risk of Mortality (i.e., death).
3. **Treatment → Lung Cancer**: Treatment can affect (either positively or negatively) the occurrence or severity of Lung Cancer.
4. **Treatment → Mortality**: Treatment has a direct effect on Mortality, which is independent of its effect on Lung Cancer.

Now, considering the question: "Does Treatment affect Mortality, considering its effect on Lung Cancer?"

The answer is: **Yes**, but with some nuance.

If we ignore the direct relationship between Treatment and Mortality (relationship 4), then:

* If Treatment reduces Lung Cancer (relationship 3), it would indirectly reduce Mortality (via reduced Lung Cancer → reduced Mortality).
* Conversely, if Treatment increases Lung Cancer, it would indirectly increase Mortality (via increa